# CCEP Preprocessing

Load in re-structured data (created in `createStimTrialEpochs.m`) and preprocesses the CCEPs.


---
> Justin Campbell & Krista Wahlstrom  
> Version: 3/05/2024

### 1. Setup

In [ ]:
# Import libraries
import os
import mne
import sys
import glob
import scipy.io
import numpy as np
import pandas as pd
import seaborn as sns
import matplotlib.pyplot as plt
from scipy.signal import savgol_filter
from scipy.signal import iirnotch, lfilter, sosfilt, butter

# Load utility functions
sys.path.insert(0, 'MNE Utils')
from prepro_utils import select_BLAES_macros, create_bipolar_montage

# Notebook settings
%matplotlib inline
%config InlineBackend.figure_format='retina'

### 2. Import Data
Load the `*StimEpochs.mat` file generated by `createStimTrialEpochs.m`.

In [ ]:
# Define file to analyze
pID = 'UIC202208'
stimPair = 'RAMG2-RAMG3'

In [ ]:
# Define datapath and find datafiles
#datapath = '/Users/justincampbell/Library/CloudStorage/Box-Box/INMANLab/BCI2000/BLAES Aim 2.1/CCEPs/Data/' # Justin's path
datapath = '/Users/inmanlab/Library/CloudStorage/Box-Box/INMANLab/BCI2000/BLAES Aim 2.1/CCEPs/Data' #Krista's path

if pID[0:3] == 'UIC':
    fileType = 'Utah'
    datapath = os.path.join(datapath, 'Utah_Data')
else:
    fileType = 'WashU'
    datapath = os.path.join(datapath, 'WashU_Data')

# Find all files for this patient
dataFile = glob.glob(os.path.join(datapath, pID) + '/' + pID + '_' + stimPair + '*.mat')
filepath = dataFile[0]

# Create Prepro folder if it does not already exist
preproDataPath = os.path.join(datapath, 'Prepro')
if not os.path.exists(preproDataPath):
    os.mkdir(preproDataPath)

In [ ]:
# Load data
mat = scipy.io.loadmat(filepath, simplify_cells = True)
data = mat['epochData'] # samples x channels x trials (± 900ms peri-stim onset)
chanNames = mat['chanNames']
fs = mat['fs']

### 3. Data Processing
- Select macroelectrode channels
- Convert from uV to V
- Create `MNE` objects
- Bipolar re-reference
- 60/120 Hz notch filter
- Manually reject bad channels ± epochs

In [ ]:
def find_macro_idxs(pID, chans):

    # Use helper function from prepro_utils.py
    macro_chan_idxs = select_BLAES_macros(chans)
    
    # Special cases (remove non sEEG channels)
    if pID == 'BJH025':
        bad_chan_idxs = [i for i, s in enumerate(chans) if s in ['FP1', 'P3', 'O1', 'FP2', 'P4', 'O2', 'P7', 'P8', 'F9', 'F10']]
        macro_chan_idxs = [i for i in macro_chan_idxs if i not in bad_chan_idxs]
        
    elif pID == 'BJH019':
        bad_chan_idxs = [i for i, s in enumerate(chans) if s in ['fp1', 'f3', 'c3', 'p3', 'o1', 'f7', 't7', 'p7', 'f9', 'fp2', 'f4', 'p4', 'o2', 'f8', 'p8', 'f10']]
        macro_chan_idxs = [i for i in macro_chan_idxs if i not in bad_chan_idxs]
        
    return macro_chan_idxs

In [ ]:
# Get macroelectrode indices for each patient
macro_idxs = find_macro_idxs(pID, chanNames)
macro_labels = chanNames[macro_idxs]

# Filter data to include only macroelectrode channels
data = data[:, macro_idxs, :]
data = data[:-1,:,:]

# Convert from uV to V
data = data / 1e6

# Reshape data
data = np.ravel(data.transpose(1,2,0))
data = data.reshape(len(macro_idxs), -1)

# Add pseudo-sample to prevent epoch cutoff
data = np.append(data, np.zeros((len(macro_idxs), 1)), axis = 1)

# Create MNE info and raw objects
info = mne.create_info(ch_names = chanNames[macro_idxs].tolist(), ch_types = 'seeg', sfreq = fs)
macros = mne.io.RawArray(data, info)

# Visualize data to reject bad channels
macros.plot()

In [ ]:
# Bipolar re-reference
anodes, cathodes = create_bipolar_montage(macro_labels, display = True) 

#Eliminate duplicate channels (issue for BJH025 in particular)
anodes_clean = []
cathodes_clean = []

for x in anodes:
    if x not in anodes_clean:
        anodes_clean.append(x)
for x in cathodes:
    if x not in cathodes_clean:
        cathodes_clean.append(x)

macrosBIP = mne.set_bipolar_reference(macros.pick(np.unique(anodes_clean+cathodes_clean)), anode = anodes_clean, cathode = cathodes_clean, drop_refs = True)
chanNamesBIP = macrosBIP.ch_names

# Create list of events
events = mne.make_fixed_length_events(macrosBIP, id = 1, start = 0, duration = 1.8)
        
# Re-create epochs using MNE
epochs = mne.Epochs(macrosBIP, events, tmin=0, tmax = 1.8, baseline=None, preload=True)

#### FILTERING
# Separate pre- and post- data
preData = epochs.copy().crop(tmin = 0, tmax = 0.9).get_data()
postData = epochs.copy().crop(tmin = 0.9).get_data()

# Create 60/120 Hz notch filters
b60, a60 = iirnotch(w0 = 60, Q = 25, fs = fs)
b120, a120 = iirnotch(w0 = 120, Q = 25, fs = fs)

# Create bandpass filter
sosButter = butter(N = 7, Wn = [0.1, 150], fs = fs, btype = 'bandpass', output = 'sos')

# Apply CAUSAL filters towards stim (pre-)
preData = lfilter(b60, a60, preData, axis = 2) # 60 Hz notch
preData = lfilter(b120, a120, preData, axis = 2) # 120 Hz notch
preData = sosfilt(sosButter, preData, axis = 2) # 7th order butterworth bandpass

# Apply CAUSAL filters towards stim (post-)
postData = np.fliplr(postData)
postData = lfilter(b60, a60, postData, axis = 2) # 60 Hz notch
postData = lfilter(b120, a120, postData, axis = 2) # 120 Hz notch
postData = sosfilt(sosButter, postData, axis = 2) # 7th order butterworth bandpass
postData = np.fliplr(postData)

# Concatenate pre- and post- data
procData = np.concatenate((preData, postData), axis = 2)

# Re-create MNE epochs
epochs = mne.EpochsArray(procData, epochs.info, events)

# Visualize to reject epochs
epochs.plot()

#### 3.1 Export Processed Data

In [ ]:
savestr = pID + "_" + stimPair
if not os.path.exists(os.path.join(preproDataPath, savestr)):
    os.mkdir(os.path.join(preproDataPath, savestr))

# Get processed data, list of rejected epochs, list of bad_chans
drop_epochs = [n for n, dl in enumerate(epochs.drop_log) if len(dl)]
events_mask = np.ones(events.shape[0], dtype = bool)
events_mask[drop_epochs] = False
keep_events = events[events_mask]
drop_chans = epochs.info['bads']
    
# Export dropped epochs, dropped chans, events, and channel labels to .csv files
np.save(os.path.join(preproDataPath, savestr, ('PreproData')), epochs.get_data())
np.save(os.path.join(preproDataPath, savestr, ('Events')), keep_events)
pd.DataFrame(drop_epochs, columns = ['Dropped Epochs']).to_csv(os.path.join(preproDataPath, savestr, 'DroppedEpochs.csv'))
pd.DataFrame(drop_chans, columns = ['Dropped Chans']).to_csv(os.path.join(preproDataPath, savestr, 'DroppedChans.csv'))
pd.DataFrame(epochs.ch_names, columns = ['Chan']).to_csv(os.path.join(preproDataPath, savestr, 'ChanLabels.csv'))